In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [18]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(train.head())

  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  


In [19]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None


In [20]:
print(train.describe())

               Age   RoomService     FoodCourt  ShoppingMall           Spa  \
count  8514.000000   8512.000000   8510.000000   8485.000000   8510.000000   
mean     28.827930    224.687617    458.077203    173.729169    311.138778   
std      14.489021    666.717663   1611.489240    604.696458   1136.705535   
min       0.000000      0.000000      0.000000      0.000000      0.000000   
25%      19.000000      0.000000      0.000000      0.000000      0.000000   
50%      27.000000      0.000000      0.000000      0.000000      0.000000   
75%      38.000000     47.000000     76.000000     27.000000     59.000000   
max      79.000000  14327.000000  29813.000000  23492.000000  22408.000000   

             VRDeck  
count   8505.000000  
mean     304.854791  
std     1145.717189  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%       46.000000  
max    24133.000000  


In [21]:
print(train.isnull().sum())

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


In [22]:
# Preencher valores 
numeric_columns = train.select_dtypes(include=[np.number]).columns
train[numeric_columns] = train[numeric_columns].fillna(train[numeric_columns].mean())
test[numeric_columns] = test[numeric_columns].fillna(test[numeric_columns].mean())

In [23]:
categorical_columns = train.select_dtypes(include=[object]).columns
train[categorical_columns] = train[categorical_columns].fillna(train[categorical_columns].mode().iloc[0])
test[categorical_columns] = test[categorical_columns].fillna(test[categorical_columns].mode().iloc[0])

C:\Users\cezar\AppData\Local\Temp\ipykernel_5908\3752423340.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train[categorical_columns] = train[categorical_columns].fillna(train[categorical_columns].mode().iloc[0])
C:\Users\cezar\AppData\Local\Temp\ipykernel_5908\3752423340.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test[categorical_columns] = test[categorical_columns].fillna(test[categorical_columns].mode().iloc[0])


In [24]:
# Manter a coluna 'PassengerId' para a submissão
passenger_ids = test['PassengerId']

In [25]:
# Converter em numéricos
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# Garantir alinhamento
train, test = train.align(test, join='left', axis=1)

# Preencher valores após o alinhamento
test.fillna(0, inplace=True)


In [26]:
# Verificar as colunas do treinamento
print("Colunas no DataFrame de treinamento:")
print(train.columns)

Colunas no DataFrame de treinamento:
Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Transported', 'PassengerId_0001_01',
       ...
       'Name_Zinopus Spandisket', 'Name_Zinor Axlentindy',
       'Name_Zinor Proorbeng', 'Name_Zinoth Lansuffle', 'Name_Zosmark Trattle',
       'Name_Zosmark Unaasor', 'Name_Zosmas Ineedeve',
       'Name_Zosmas Mormonized', 'Name_Zubeneb Flesping',
       'Name_Zubeneb Pasharne'],
      dtype='object', length=23741)


In [11]:
# Verificar se 'Transported' está presente
if 'Transported' in train.columns:
    print("Sim")
else:
    print("Não")


Sim


In [27]:
if 'Transported' in train.columns:
    X = train.drop('Transported', axis=1)
    y = train['Transported']

    # Dados de treinamento e validação
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Padronizar os dados
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    
    # Remover a coluna 'Transported'
    X_test = test.drop('Transported', axis=1)
    X_test = scaler.transform(X_test)

    # Treinar o modelo de classificação Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Prever 
    y_pred = model.predict(X_val)

    # Avaliar 
    accuracy = accuracy_score(y_val, y_pred)
    print(f'Acurácia: {accuracy:.2f}')
else:
    print("Verifique os dados faltantes!")


Acurácia: 0.78


In [28]:

if 'Transported' in train.columns:
    test_predictions = model.predict(X_test)

In [30]:
# Fazer previsões no conjunto de teste, se a coluna 'Transported' estiver presente
if 'Transported' in train.columns:
    test_predictions = model.predict(X_test)

    # Criar o arquivo de submissão
    submission = pd.DataFrame({'PassengerId': passenger_ids, 'Transported': test_predictions})
    submission.to_csv('submission.csv', index=False)
    print("Criado!")
else:
    print("Não criado")

Criado!
